In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
GMARKET_BEST_URL = "http://corners.gmarket.co.kr/Bestsellers"

`Response [200]` 확인

In [ ]:
response = requests.get(GMARKET_BEST_URL)
response

<Response [200]>

`BeautifulSoup`으로 parsing

In [ ]:
dom = BeautifulSoup(response.content, 'html.parser')

일단 한 개의 아이템만 크롤링 해 본다.

`!pip install beautifulsoup4==4.8.1`

In [ ]:
best_item1 = dom.select_one("#gBestWrap > div > div:nth-child(5) > div:nth-child(3) > ul > li:nth-child(1)")
best_item1

<li class="first">
<p class="no1" id="no1">1</p>
<div class="thumb">
<a href="http://item.gmarket.co.kr/Item?goodscode=1537492125&amp;ver=637743928094188364" onclick="pdsClickLog('200000680', 'Item', {'ASN': 1, 'goodsCode': '1537492125'});"><img alt="[아트릭스]아트릭스 핸드크림 75mlX4개+립밤 / 쿠폰가 7670원" class="lazy" data-original="http://gdimg.gmarket.co.kr/1537492125/still/300?ver=2021120613" height="210" src="http://image.gmarket.co.kr/challenge/neo_image/no_image.gif" width="210"/></a>
</div>
<!--div class="goods-view">
									<a href="http://minishop.gmarket.co.kr/nivea01"><span class="view">판매자 다른상품 보기</span>바이어스도르프 공식</a>
								</div-->
<a class="itemname" href="http://item.gmarket.co.kr/Item?goodscode=1537492125&amp;ver=637743928094188364" onclick="pdsClickLog('200000680', 'Item', {'ASN': 1, 'goodsCode': '1537492125'});">[아트릭스]아트릭스 핸드크림 75mlX4개+립밤 / 쿠폰가 7670원</a>
<div class="item_price">
<div class="o-price"><span><span>35,400원</span></span></div>
<div class="s-price">
<strong><span><span

`nth-child`를 쓰지 않고 하려면..?

In [ ]:
g_best_top = dom.select("#gBestWrap > .gbest-top > div")
best_list = g_best_top[2].select("div.best-list")
best_list = best_list[1]

item_best1 = best_list.select("li")[0]
item_best1

<li class="first">
<p class="no1" id="no1">1</p>
<div class="thumb">
<a href="http://item.gmarket.co.kr/Item?goodscode=1537492125&amp;ver=637743928094188364" onclick="pdsClickLog('200000680', 'Item', {'ASN': 1, 'goodsCode': '1537492125'});"><img alt="[아트릭스]아트릭스 핸드크림 75mlX4개+립밤 / 쿠폰가 7670원" class="lazy" data-original="http://gdimg.gmarket.co.kr/1537492125/still/300?ver=2021120613" height="210" src="http://image.gmarket.co.kr/challenge/neo_image/no_image.gif" width="210"/></a>
</div>
<!--div class="goods-view">
									<a href="http://minishop.gmarket.co.kr/nivea01"><span class="view">판매자 다른상품 보기</span>바이어스도르프 공식</a>
								</div-->
<a class="itemname" href="http://item.gmarket.co.kr/Item?goodscode=1537492125&amp;ver=637743928094188364" onclick="pdsClickLog('200000680', 'Item', {'ASN': 1, 'goodsCode': '1537492125'});">[아트릭스]아트릭스 핸드크림 75mlX4개+립밤 / 쿠폰가 7670원</a>
<div class="item_price">
<div class="o-price"><span><span>35,400원</span></span></div>
<div class="s-price">
<strong><span><span

In [ ]:
item_best1.select_one("a.itemname").text

'[아트릭스]아트릭스 핸드크림 75mlX4개+립밤 / 쿠폰가 7670원'

In [ ]:
item_best1.select_one("div.item_price > div.s-price span").text

'11,800원'

`goodscode` 찾기
- href에 들어있는 `goodscode`를 찾아보자
- `attribute`에 있는 값을 가져오기 위해서는 `get(attribute name)`

In [ ]:
link_item1 = item_best1.select_one("a.itemname").get("href")
print(link_item1)

http://item.gmarket.co.kr/Item?goodscode=1537492125&ver=637743928094188364


In [ ]:
params = link_item1.split("?")
params = params[1]
param_list = params.split("&")
goodscode_param = param_list[0]
goodscode = goodscode_param.split("=")[1]
goodscode

'1537492125'

`urllib` 라이브러리
- 기본적인 웹 통신 및 url을 관리하기 위한 여러 기능들이 들어있는 모듈
- `urlparse`, `parse_qs`

In [ ]:
from urllib.parse import urlparse, parse_qs

In [ ]:
# urlparse
parsed_url = urlparse(link_item1)
parsed_url

ParseResult(scheme='http', netloc='item.gmarket.co.kr', path='/Item', params='', query='goodscode=1537492125&ver=637743928094188364', fragment='')

In [ ]:
url_query = parsed_url.query
url_query

'goodscode=1537492125&ver=637743928094188364'

In [ ]:
# parse_qs url의 쿼리 부분을 딕셔너리 형태로 바꿔준다.
parsed_query = parse_qs(url_query)
parsed_query

{'goodscode': ['1537492125'], 'ver': ['637743928094188364']}

In [ ]:
item1_goodscode = parsed_query['goodscode'][0]
item1_goodscode

'1537492125'

`image` url 가져오기

In [ ]:
img_url = item_best1.select_one("div.thumb > a > img").get("data-original")
img_url

'http://gdimg.gmarket.co.kr/1537492125/still/300?ver=2021120613'

`img` 저장 라이브러리 - `urllib.request.urlretrieve`

In [ ]:
from urllib.request import urlretrieve

# urlretrieve(이미지 링크, 저장할 파일명)
 # 임의로 png라고 적어준다. 이미지 확장자가 있으면 그냥 그거 그대로 쓰는게 좋음.
urlretrieve(img_url, item1_goodscode+".png")

('1537492125.png', <http.client.HTTPMessage at 0x1853b8bfdc0>)

모든 아이템 가져오기

In [ ]:
!mkdir gmarket_images

In [ ]:
for item in best_list.select("li"):
    item_name  = item.select_one("a.itemname").text
    item_price = item.select_one("div.item_price > div.s-price span").text
    item_link  = item.select_one("a.itemname").get("href")
    
    parsed_url = urlparse(item_link)
    parsed_query = parse_qs(parsed_url.query)
    item_code = parsed_query['goodscode'][0]
    
    img_url = item.select_one("div.thumb > a > img").get("data-original")
    urlretrieve(img_url, "./gmarket_images/"+item_code+".png")